In [10]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexing_distinct_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)

In [11]:
def get_accuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [12]:
indexing_distinct_tables.indices.close(index='distinct_tables')
indexing_distinct_tables.indices.put_settings(index='distinct_tables', body={"index": {"similarity": {"default": {"type": "BM25"}}}})
indexing_distinct_tables.indices.open(index='distinct_tables')

{'acknowledged': True, 'shards_acknowledged': True}

In [13]:
def search_indexing(query, tableField, topK):
    
    result= indexing_distinct_tables.search(
        index="distinct_tables", 
        body = {
        "_source": ["tablePgID"],
        "from" : 0,
        "size" : topK,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tablePgTitle"] 
              #"fields": ["tablePgTitle","tableHeader","tableBody"] 
              #"fields": [tableField]
            }
        }
    })
    
    return result

In [14]:
articles = pd.read_csv('dataset/data_articles_test.csv', delimiter=',')

In [15]:
def execute_search(tableField, topK):

    accuracy = []

    for i,row in (articles.iterrows()):
    
        articl_key = row['page_id']
        article_title = row['page_title']
        article_meta_description = row['meta_description']
        
        #catch = article_title+" "+article_meta_description
            
        result = search_indexing(article_title,tableField,topK)
    
        idRankedTables = []
    
        for hit in result['hits']['hits']:
    
            tablePgId = hit['_source']['tablePgID']
    
            idRankedTables.append(tablePgId)
    
        accuracy.append(get_accuracy(idRankedTables,articl_key))

    print(str(round(np.mean(accuracy),4)))

In [39]:
tableField = ["tablePgTitle"]
#tableField = ["tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"]
topK = [100]

In [17]:
# for k in topK:
    
#     print("")
    
#     for field in tableField:
        
#         print("Acc@"+str(k)+" - "+field)
        
#         execute_search(field,k)

In [40]:
for k in topK:
    
    print("Acc@"+str(k))
    
    execute_search("",k)

Acc@100
0.46
